## Making an Auto-eval of final statement, using GPT-4

##### Remarks about performance

- Tested on 20 random samples from training data (statement + input story)-pairs, each asking if 13 critiquing principles are answered -> 260 tested principles in total.
- Autoeval classification of whether a principle is broken or not did vary a bit from run to run - so temperature needed to be low.
- My annotation of number of broken principles per statement is:
  5, 5, 3, 4, 6 __ 4, 4, 3, 3, 5 __ 4, 3, 4, 3, 3 __ 6, 1, 1, 3, 1
- Autoeval made the following evaluation of the same statements: 
  4, 3, 3, 5, 6 __ 4, 4, 2, 4, 5 __ 3, 3, 5, 3, 3 __ 7, 2, 1, 2, 1 
- As these results show, autoeval is typically just one penalty-point away (thinks there is one more or less broken principle), compared to my annotations. On one occasion autoeval was 2 points away.
- In 10/20 evals, it gives a different number of penalty points. 5 times it is higher, and 5 times it is lower.
- It usually catches the same principles as my annotations, but can sometimes mix up parts of which principles are broken or not.. The most common principles it failed to register as broken was amends (5 times) and improve+prevent (4 times). The most common principles it "added" (classified as broken when hey were not) was emotions, charater and hallucination (3 times each).
- Also tested a bit using GPT-3.5-turbo, which performed worse

In [1]:
from datasets import load_dataset
from key import test_key, miri_key
from info import principles, whatCR, rewriteR, yesnoCR_turned
import openai
import numpy as np
import time

openai.api_key = miri_key

# Train stories
input_storyR = "My name is Ronald Smith and I hit a guy. I am 32 years old and have never done anything like this. I had been drinking with some friends and then this guy said something very rude to my friend, so I got really angry and lost control for a moment."
input_storyM = "Hi, I’m Muhammad Hussain and unfortunately, I need to plead guilty for spitting in a man’s face. I was on my way home from work when I got some derogatory comments on the street. I was having a particularly bad day after losing a big client at work (I’m a lawyer), and so I got really angry. I talked back to the guy, but he kept nagging at me, and I ended up spitting in his face. I’m really sorry about it and I wish I could take it back. I would actually love to somehow gain better control of my anger by attending an anger management program. I am 32 years old, have never done anything criminal, and I am an eager member of the Muslim community in town, where I voluntarily help out about twice a week."
input_storyA = "I kicked a football through a storefront window. It was the middle of the day and I was walking home from soccer practice when I lost control of the ball and accidentally hit the window. There were a few people inside, but luckily no one was hurt. I am 19 years old and have never done anything criminal before. I feel so bad about this. And my name is Abbey Davies. I also have a really cute dog named Ollie."
input_storyW = "My name is Charlotte Wright and I am pleading guilty to drunk driving. My 8-year-old child was supposed to sleep over at a friend’s house but called me and wanted to go home after dinner. I had already had a few glasses of wine but did not consider that when getting in my car to go pick up my son. Frankly, I am really embarrassed by this incident, and incredibly remorseful, as I understand the severe consequences it could have had – not only for me and my son, but for others on the road as well. I already have 3 penalty points on my driver’s licence for speeding, but my son depends on me driving him to school every day, so I want to see if it is possible to not lose my licence. I’m 43 years old and don’t have anything else on my record."
input_storyJ = "My name is James Campbell. I run a small bike shop in town and I am guilty of tax evasion. My business has not been doing very well the past 2 years, and taxes became an expense I felt like I could not afford. So I didn’t register the work I did that was paid in cash, and did not provide receipts or put it in my books. I’m not sure of the exact amount, but I’m guessing I have failed to register about 7000 pounds. I also have a previous conviction of common assault. I do feel really bad about it and hope I will be able to make amends somehow. I have also decided to put down the business and apply for a steadier job to prevent anything like this from happening again.  I’m 48 years old."
input_story = input_storyW

data_path = "/home/vsb29/rds/hpc-work/project/data/data_file1.json"
training_data = load_dataset("json", data_files=data_path)['train']

Found cached dataset json (/home/vsb29/.cache/huggingface/datasets/json/default-ba6b1c046a561edb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [99]:
random_checks = np.random.randint(low=0, high=281, size=5)
print(random_checks)
random_checks1 = np.array([191, 128, 254, 180, 278])
random_checks2 = np.array([30, 268, 275, 247, 107])
random_checks3 = np.array([131, 90, 195, 51, 103])
random_checks4 = np.array([122, 233, 60, 272, 197])
random_checks = random_checks1

[156  28 259 123  59]


In [100]:
for i in random_checks:
    print(training_data['input'][int(i)].split('### STATEMENT: ')[0])

### INPUT STORY: My name is Charlotte Wright and I am pleading guilty to drunk driving. My 8-year-old child was supposed to sleep over at a friend’s house but called me and wanted to go home after dinner. I had already had a few glasses of wine but did not consider that when getting in my car to go pick up my son. Frankly, I am really embarrassed by this incident, and incredibly remorseful, as I understand the severe consequences it could have had – not only for me and my son, but for others on the road as well. I already have 3 penalty points on my driver’s licence for speeding, but my son depends on me driving him to school every day, so I want to see if it is possible to not lose my licence. I’m 43 years old and don’t have anything else on my record.

### INPUT STORY: My name is Ronald Smith and I hit a guy. I am 32 years old and have never done anything like this. I had been drinking with some friends and then this guy said something very rude to my friend, so I got really angry an

In [101]:
statements = [training_data['input'][int(i)].split('### STATEMENT: ')[1] for i in random_checks]
stories = [training_data['input'][int(i)].split('### STATEMENT: ')[0].strip('### INPUT STORY: ') for i in random_checks]
print(stories)
print(statements)

['My name is Charlotte Wright and I am pleading guilty to drunk driving. My 8-year-old child was supposed to sleep over at a friend’s house but called me and wanted to go home after dinner. I had already had a few glasses of wine but did not consider that when getting in my car to go pick up my son. Frankly, I am really embarrassed by this incident, and incredibly remorseful, as I understand the severe consequences it could have had – not only for me and my son, but for others on the road as well. I already have 3 penalty points on my driver’s licence for speeding, but my son depends on me driving him to school every day, so I want to see if it is possible to not lose my licence. I’m 43 years old and don’t have anything else on my record.\n', 'My name is Ronald Smith and I hit a guy. I am 32 years old and have never done anything like this. I had been drinking with some friends and then this guy said something very rude to my friend, so I got really angry and lost control for a moment.

In [102]:
evaluate_assertions = "Below you will see a list of general assertions followed by a statement, and an input story. Consider the input story and statement, and go through one by one assertion, discussing if this statement and input story pair holds or breaks that assertion. Output this as a list in the same order as the assertions."
assertions = "- The statement is well-written, non-repetative, and easy to follow.\n- The statement mentions that the defendant is pleading guilty.\n- The statement does not contain any completely irrelevant information (eg it does not mention the 'input story' or another statement).\n- The statement only says that the defendant has suffered, if this is stated by the input story. If the input story implies that the defendant has suffered as a consequence of the offense, the statement should include this. Great emotional suffering is not mentioned in the statement.\n- If the defendant's age is below 30, or above 79, it is included in the statement. If the defendant's age is between 30-79, the age is not mentioned in the statement.\n- If the input story implies that the defendant has a clean record, this is mentioned in the statement. The statement does imply that the defendant has a clean record, if the input story cannot support this.\n- If the input story states that the defendant has a mental illness or disability, this is included in the statement. If the input story says nothing about a mental illness or disability, no such thing is mentioned in the statement either.\n- The statement properly explains what the offense is and how it happened - as in the input story. It does not add or skip important parts from the input story.\n- If the input story states that the defendant wants to or has already made amends; this is included in the statement (including examples, if this is present in the input story). If amends (eg compensating the victim) is not mentioned in the input story, it is not mentioned in the statement.\n- If the input story states that the defendant wants to or has already made efforts to self-improve or prevent the offense from happening again; this is included in the statement (including examples, if this is present in the input story). If the input story does not mention self-improvement (eg counselling) or preventing repetition of the crime, it is not mentioned in the statement either.\n- If the input story gives information that speaks to the defendant's good character, the statement includes this (including examples, if this is present in the input story). If no character-building information (eg voluntary work or contributing to society) is given in the input story, it is not mentioned in the statement either.\n- The emotions in the statement is the same as the emotions stated in the input story. Eg the statement does not show more regret, guilt or anger than the input story.\n- All information found in the statement is supported by information in the input story."
get_penalties = "Based on these evaluations - how many assertions are broken for our input story and statement pair?"
diff = "Make sure to think carefully about what is said in the input story VS what is said in the statement."

# evaluate_assertions = "Below you will see a list of general assertions about a statement, and an input story. Consider the input story and statement that will follow the assertions, and go through one by one assertion, discussing if this statement and input story pair holds or breaks that assertion. Output this as a list in the same order as the assertions."
# assertions = "- The statement is well-written, non-repetative, and easy to follow.\n- The statement mentions that the defendant is pleading guilty.\n- The statement does not contain any completely irrelevant information (eg mentioning the 'input story' or another statement).\n- The statement only says that the defendant has suffered, if this is stated by the input story. If the input story implies that the defendant has suffered as a consequence of the offense, the statement should include this. Great emotional suffering is not mentioned in the statement.\n- If the defendant's age is below 30, or above 79, it is includede in the statement. If the defendant's age is between 30-79, the age is not mentioned in the statement.\n- If the input story implies that the defendant has a clean record, this is mentioned in the statement. The statement does imply that the defendant has a clean record, if the input story cannot support this.\n- If the input story states that the defendant has a mental illness or disability, this is included in the statement. If the input story says nothing about a mental illness or disability, no such thing is mentioned in the statement either.\n- The statement properly explains what the offense is and how it happened - as in the input story. It does not add or skip important parts from the input story.\n- If the input story states that the defendant wants to or has already made amends; this is included in the statement. If amends (eg compensating the victim) is not mentioned in the input story, it is not mentioned in the statement.\n- If the input story states that the defendant wants to or has already made efforts to self-improve or prevent the offense from happening again; this is included in the statement. If the input story does not mention self-improvement (eg counselling) or preventing repetition of the crime, it is not mentioned in the statement either.\n- If the input story gives information that speaks to the defendant's good character, the statement includes this. If no character-building information (eg voluntary work or contributing to society) is given in the input story, it is not mentioned in the statement either.\n- The emotions portrayed in the statement is an accurate reflection of the emotions stated in the input story.\n- All information found in the statement is supported by information in the input story."

answ1 = ['']*5
for i in range(5):
    chat_log = [{"role": "system", "content": evaluate_assertions + diff},
                {"role": "user", "content": assertions + '\n' + "### INPUT STORY: " + stories[i] + '\n' + "\n### STATEMENT: " + statements[i]}]

    answ1[i] = openai.ChatCompletion.create(
        #model="gpt-3.5-turbo-0613",
        model="gpt-4-0613",
        messages=chat_log,
        temperature=0.1,
        top_p = 0.9,
        )['choices'][0]['message']['content'].strip()

##### Checking results

In [111]:
answn = answ1.copy()

In [118]:
# Checking results
i=3
print(i)
print(stories[i])
print(statements[i])
print('\n')
print(answn[i])

3
My name is Charlotte Wright and I am pleading guilty to drunk driving. My 8-year-old child was supposed to sleep over at a friend’s house but called me and wanted to go home after dinner. I had already had a few glasses of wine but did not consider that when getting in my car to go pick up my son. Frankly, I am really embarrassed by this incident, and incredibly remorseful, as I understand the severe consequences it could have had – not only for me and my son, but for others on the road as well. I already have 3 penalty points on my driver’s licence for speeding, but my son depends on me driving him to school every day, so I want to see if it is possible to not lose my licence. I’m 43 years old and don’t have anything else on my record.

Your Honor, I, Charlotte Wright, stand before you today to firmly express my recognition and acceptance of the wrongfulness of my actions, and emphasize the significant role that my guilty plea plays as a crucial mitigating factor in this case. By pl

In [122]:
print("Statement nr ", i)
nr_to_princ = {0: 'coherence', 1:'guilty plea', 2:'irrelevant', 3:'suffering', 4:'age', 5:'record', 6:'mental', 7:'explain', 8:'amends', 9:'improve + prevent', 10:'character', 11:'emotions', 12:'hallucination'}
per = answn[i].strip('- ').split("\n- ")
broken = 0
alle = 0
still_none = []
for t, j in enumerate(per):
    alle += 1
    if "True" in j and "False" in j:
        print("BOTH", j)
    elif 'False' in j:
        broken += 1
        print("broken princ:", nr_to_princ[t])
    elif not ("True" in j or "False" in j):
        #print(t, "none")
        if ': Yes' in j and ': No' in j:
            print('both Y and N')
        elif ': Yes' in j or ': not applicable' in j.lower():
            continue
        elif ': No' in j:
            broken += 1
            print("Broken YN: ", nr_to_princ[t])
        elif not (": Yes" in j or ": No" in j):
            if ": The statement holds this assertion" in j or ": This assertion holds" in j:
                continue
            elif ": The statement breaks this assertion" in j:
                broken += 1
                print("Broken YN: ", nr_to_princ[t])
            else:
                if ": Breaks" in j:
                    broken += 1
                    print("Broken YN: ", nr_to_princ[t])
                elif ": Holds" in j:
                    continue
                else:
                    print("Still none:", nr_to_princ[t])
                    still_none.append(t)

print(f"# broken principles: {broken}/{alle}")

Statement nr  3
Still none: coherence
# broken principles: 0/13


In [124]:
for p in still_none:
    part = answn[i].strip('- ').split("\n- ")[p]
    print(part)

    chat_log = [{"role": "system", "content": "Below you will see a claim, and then a discussion of the claim. The structure of this is as follows \nclaim: discussion\nBased on the discussion, would you say that the claim is satisfied or not?"},
                {"role": "user", "content": part}]

    response = openai.ChatCompletion.create(
        #model="gpt-3.5-turbo-0613",
        model="gpt-4-0613",
        messages=chat_log,
        temperature=0.1,
        top_p = 0.9,
        )['choices'][0]['message']['content'].strip()

    print(response)

The statement is well-written, non-repetative, and easy to follow: The statement is well-written and easy to follow, but it is somewhat repetitive, particularly in emphasizing the defendant's guilty plea and remorse.
The claim is not satisfied.


In [114]:
#Should be
# My annotations     : 5, 5, 3, 4, 6 __ 4, 4, 3, 3, 5 __ 4, 3, 4, 3, 3 __ 6, 1, 1, 3, 1
# Oldest_with_changes: 4, 3, 3, 5, 6 __ 4, 4, 2, 4, 5 __ 3, 3, 5, 3, 3 __ 7, 2, 1, 2, 1 


In [117]:
for pp in range(5):
    i = pp
    print("Statement nr ", i)
    nr_to_princ = {0: 'coherence', 1:'guilty plea', 2:'irrelevant', 3:'suffering', 4:'age', 5:'record', 6:'mental', 7:'explain', 8:'amends', 9:'improve + prevent', 10:'character', 11:'emotions', 12:'hallucination'}
    per = answn[i].strip('- ').split("\n- ")
    broken = 0
    alle = 0
    for t, j in enumerate(per):
        alle += 1
        if "True" in j and "False" in j:
            print("BOTH", j)
        elif 'False' in j:
            broken += 1
            print("broken princ:", nr_to_princ[t])
        elif not ("True" in j or "False" in j):
            #print(t, "none")
            if ': Yes' in j and ': No' in j:
                print('both Y and N')
            elif ': Yes' in j or ': not applicable' in j.lower():
                continue
            elif ': No' in j:
                broken += 1
                print("Broken YN: ", nr_to_princ[t])
            elif not (": Yes" in j or ": No" in j):
                if ": The statement holds this assertion" in j or ": This assertion holds" in j:
                    continue
                elif ": The statement breaks this assertion" in j:
                    broken += 1
                    print("Broken YN: ", nr_to_princ[t])
                else:
                    if ": Breaks" in j:
                        broken += 1
                        print("Broken YN: ", nr_to_princ[t])
                    elif ": Holds" in j:
                        continue
                    else:
                        print("Still none:",  nr_to_princ[t])

    print(f"# broken principles: {broken}/{alle}")
    print()

Statement nr  0
Broken YN:  age
Broken YN:  amends
Broken YN:  improve + prevent
Broken YN:  emotions
Broken YN:  hallucination
# broken principles: 5/13

Statement nr  1
broken princ: explain
broken princ: improve + prevent
broken princ: emotions
broken princ: hallucination
# broken principles: 4/13

Statement nr  2
broken princ: amends
broken princ: character
broken princ: emotions
broken princ: hallucination
# broken principles: 4/13

Statement nr  3
Still none: coherence
# broken principles: 0/13

Statement nr  4
Broken YN:  suffering
Broken YN:  age
Broken YN:  record
Broken YN:  character
Broken YN:  emotions
Broken YN:  hallucination
# broken principles: 6/13



In [21]:
# This type of eval often gives the wrong result... We do it by hand instead

# chat_log = [{"role": "system", "content": evaluate_assertions},
#             {"role": "user", "content": assertions + '\n' + "### INPUT STORY: " + input_story + '\n' + "\n### STATEMENT: " + initial_statement},
#             {"role": "assistant", "content": answ4['choices'][0]['message']['content'].strip()},
#             {"role": "user", "content": get_penalties}]

# pens4 = openai.ChatCompletion.create(
#             #model="gpt-3.5-turbo-0613",
#             model="gpt-4-0613",
#             messages=chat_log,
#             temperature=0.1,
#             top_p = 0.9,
#         )

# pens4['choices'][0]['message']['content'].strip()

In [ ]:
print(sto)
print(statements[2])
print()

In [73]:
for j in range(5):
    print(stories[j])
    print(statements[j])
    print()
    print("________________________________________________")

My name is Ronald Smith and I hit a guy. I am 32 years old and have never done anything like this. I had been drinking with some friends and then this guy said something very rude to my friend, so I got really angry and lost control for a moment.

Your Honor, I plead guilty to the offense of hitting someone in a moment of anger. I am deeply sorry for my actions and recognize that they were completely unacceptable. I would like to express my regret for what happened on that night and acknowledge the harm caused to the victim. I understand that this was an isolated incident, and while this does not justify my behavior, it is important to note that it was a momentary lapse of judgment triggered by provocation. However, I take full responsibility for my actions. As someone who has never been involved in any criminal activity before this incident, I am willing to cooperate with the authorities and take responsibility for my actions. My arrest and detention have taken a toll on me and my fam